<a href="https://colab.research.google.com/github/eliascorreajr/O_Bot_Cineasta/blob/main/O_Bot_Cineasta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🎬 O Bot Cineasta 🍿
Imagine um amigo entusiasta de cinema que não só conhece todos os filmes e séries, mas também consegue falar sobre eles de forma descontraída e interessante. Agora imagine que esse amigo é um bot! Esse é o "Bot Cineasta", um projeto feito em Python que usa a magia da API do TMDb (The Movie Database) para buscar informações sobre qualquer filme ou série que você possa imaginar.

🤖 Como Funciona o Bot?
Pergunte Qualquer Coisa: Você quer saber sobre "Batman"? Ou talvez algo mais nicho como "O Grande Hotel Budapeste"? Não se preocupe! Digite o nome, e o Bot Cineasta entra em ação.

Detalhes, Detalhes, Detalhes: Além do título e da sinopse, o bot te dá um toque especial — ele te diz o ano de lançamento. Agora você pode distinguir entre o "Batman" de 1989 e o de 2021, sem confusão!

Perdido em Tradução?: Se a sinopse estiver em inglês, não tem problema! O Bot Cineasta é poliglota, graças ao Google Translator, e traz tudo mastigadinho em português para você.

Criatividade a Mil: Após pegar todas essas informações, o bot não para por aí. Ele usa uma IA para reescrever a sinopse de uma maneira mais descontraída. Porque, vamos combinar, às vezes os resumos são um pouco... secos.

🔄 Loop Infinito de Diversão
O Bot Cineasta não cansa! Ele fica rodando em um loop, pronto para a próxima consulta, até você dizer "sair". Ele é o companheiro perfeito para suas noites de cinema, ajudando você a escolher o que assistir com informações detalhadas e um toque de diversão.

🎥 Em Resumo
Esse projeto transforma o que poderia ser uma busca monótona por informações em uma experiência interativa e envolvente. É como ter um amigo cinéfilo digital ao seu lado, pronto para ajudar você a navegar pelo vasto mundo do entretenimento. Então, prepare a pipoca e deixe o Bot Cineasta fazer o trabalho pesado!


In [30]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai
!pip install tmdbsimple
!pip install googletrans==4.0.0-rc1


In [32]:
import requests
import numpy as np
import tmdbsimple as tmdb
import pandas as pd
import google.generativeai as genai
from googletrans import Translator, LANGUAGES
from google.colab import userdata

In [33]:
# Configurações da API do TMDb
tmdb.API_KEY = userdata.get("CHAVE_SECRETA_TMDB")
tmdb.REQUESTS_TIMEOUT = 10

In [34]:
# Configurações da API do Google AI Studio
api_key = userdata.get("CHAVE_SECRETA_AISTUDIO")
genai.configure(api_key=api_key)


In [38]:
# Modelos de embedding e geração de texto
model_embedding = "models/embedding-001"
model_geracao = genai.GenerativeModel("gemini-1.0-pro", generation_config={"candidate_count": 1, "temperature": 0.5})

# Inicializar o tradutor
translator = Translator()

# Função para criar embeddings
def criar_embedding(titulo, conteudo):
    """Cria um embedding para um documento."""
    try:
        if conteudo is None or conteudo.strip() == "":
            return None
        return genai.embed_content(
            model=model_embedding,
            content=conteudo,
            title=titulo,
            task_type="RETRIEVAL_DOCUMENT"
        )["embedding"]
    except Exception as e:
        print(f"Erro ao criar embedding: {e}")
        return None

# Função para buscar por similaridade
def buscar_similaridade(consulta, dataframe):
    """Busca o documento mais similar à consulta."""
    try:
        embedding_consulta = genai.embed_content(model=model_embedding, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]
        produtos_escalares = np.dot(np.stack(dataframe["Embeddings"].to_numpy()), embedding_consulta)
        indice = np.argmax(produtos_escalares)
        return dataframe.iloc[indice]  # Retorna a Series correspondente à linha
    except Exception as e:
        print(f"Erro ao buscar similaridade: {e}")
        return None

# Função para gerar resposta
def gerar_resposta(resultado):
    """Gera uma resposta completa e descontraída."""
    try:
        titulo = resultado.get('Título', "Informação indisponível")
        ano = resultado.get('release_date', "Data Indisponível")[:4] if 'release_date' in resultado else "Data Indisponível"
        sinopse_original = resultado.get('Conteúdo', "Sem conteúdo disponível")
        classificacao = str(resultado.get('Classificação', "N/A"))


        if translator.detect(sinopse_original).lang == 'en':
            sinopse_original = translator.translate(sinopse_original, dest='pt').text

        prompt = f"Título: {titulo}\n\nAno: {ano}\n\nSinopse Original: {sinopse_original}\n\nClassificação: {classificacao}\n\nReescreva a sinopse de forma mais descontraída, sem adicionar novas informações:"
        resposta = model_geracao.generate_content(prompt)

        return (
            f"\n{'.' * 70}\n"  # Linha pontilhada antes
            f"Título: {titulo}\n"
            f"{'.' * 70}\n"  # Linha pontilhada depois
            f"Sinopse Original: {sinopse_original}\n"
            f"{'.' * 70}\n"
            f"Vamos lá... opinião sincera: {resposta.text}\n"
            f"{'.' * 70}\n"
            f"Classificação Geral: {classificacao}\n"
            f"{'.' * 70}\n"
            f"Ano de Lançamento: {ano}\n"
            f"{'.' * 70}\n"
        )
    except Exception as e:
        print(f"Erro ao gerar resposta: {e}")
        return "Desculpe, não consegui gerar uma resposta para sua pergunta."

# Loop principal do programa
while True:
    tipo_conteudo = input("Deseja buscar um filme, série ou pessoa? (ou 'sair' para encerrar): ").lower()

    if tipo_conteudo == 'sair':
        break

    if tipo_conteudo not in ("filme", "série", "pessoa"):
        print("Tipo de conteúdo inválido. Por favor, escolha entre 'filme', 'série' ou 'pessoa'.")
        continue

    nome = input(f"Digite o nome exato do {tipo_conteudo}: ")

    try:
        tipo_pesquisa = "movie" if tipo_conteudo == "filme" else "tv" if tipo_conteudo == "série" else "person"
        search = tmdb.Search()

        if tipo_pesquisa == "movie":
            search.movie(query=nome, language='pt-BR')
        elif tipo_pesquisa == "tv":
            search.tv(query=nome, language='pt-BR')
        else:
            search.person(query=nome)

        if not search.results:
            print("Nenhum resultado encontrado para sua pesquisa. Verifique se o nome está correto.")
            continue

        df = pd.DataFrame(search.results)
        df['ID'] = df['id']
        df['Título'] = df['title'] if 'title' in df.columns else df['name'] if 'name' in df.columns else "Título Indisponível"
        df['Conteúdo'] = df['overview'] if 'overview' in df.columns else "Sem sinopse disponível"
        df['Classificação'] = (df['vote_average'] * 10).fillna(0) if 'vote_average' in df.columns else 0
        df['Ano'] = df['release_date'].str[:4] if 'release_date' in df.columns else df['first_air_date'].str[:4] if 'first_air_date' in df.columns else "Data Indisponível"

        df["Embeddings"] = df.apply(lambda row: criar_embedding(row["Título"], row["Conteúdo"]), axis=1)
        df = df.dropna(subset=['Embeddings'])

        resultado = buscar_similaridade(nome, df)
        if resultado is not None:
            resposta_final = gerar_resposta(resultado)
            print("\nResposta:\n", resposta_final)
        else:
            print("Desculpe, não consegui encontrar uma resposta para sua pergunta.")

    except requests.exceptions.RequestException as e:
        print(f"Erro na comunicação com a API do TMDb: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")


Deseja buscar um filme, série ou pessoa? (ou 'sair' para encerrar): filme
Digite o nome exato do filme: kung fu panda 4

Resposta:
 
......................................................................
Título: Kung Fu Panda 4
......................................................................
Sinopse Original: Po está prestes a se tornar o novo líder espiritual do Vale da Paz, mas antes que possa fazer isso, ele deve encontrar um sucessor para se tornar o novo Dragão Guerreiro.  Ele parece encontrar uma em Zhen, uma raposa com muitas habilidades promissoras, mas que não gosta muito da ideia de Po treiná-la.
......................................................................
Vamos lá... opinião sincera: Prepare-se para a quarta aventura do nosso querido Po, o panda mais legal do pedaço! Desta vez, ele está prestes a virar o chefe espiritual do Vale da Paz, mas antes precisa encontrar um novo Dragão Guerreiro. E adivinha quem ele acha que é a escolhida? Zhen, uma raposa cheia de 